<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/item-based-cf/item_based_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration

In [2]:
#@title Download necessary libraries
!pip install pyspark -qq
!pip install -U -q PyDrive -qq
!apt install openjdk-8-jdk-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 122542 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headles

In [3]:
#@title Imports
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly

import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
from pyspark import SparkContext, SparkConf
from pyspark.ml.linalg import SparseVector, DenseVector, VectorUDT

from tqdm.notebook import tqdm
import time
import gc

from google.colab import drive

from typing import Tuple, Callable, Dict
from functools import reduce
import pickle

In [4]:
#@title Set up variables
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/drive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [5]:
#@title Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [6]:
drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


## Setup ngrok

In [7]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=274869b4656847dae7bf17eac459f8a3efeec2b4cf5f50d94053acafd0448361
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [8]:
!ngrok authtoken 2NVN8kdoOnMVtlDGGWtwsbT5M3Q_2EJv2HE77FEXkz978Qtnq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [9]:
from pyngrok import ngrok

# Open a ngrok tunnel on the port 4050 where Spark is running
port = '4050'
public_url = ngrok.connect(port).public_url

In [10]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://314a-34-148-172-202.ngrok-free.app" -> "http://127.0.0.1:4050"


In [11]:
#@title Check if everything is ok
spark, sc._conf.getAll()


(<pyspark.sql.session.SparkSession at 0x7fc010ee0af0>,
 [('spark.executor.extraJavaOptions',
   '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false -XX:+UseG1GC'),
  ('spark.app.submitTime', '1685812755246'),
  ('spark.app.name', 'PySparkTutorial

# Data acquisition

In [12]:
song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

playlist_schema_mapped = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", VectorUDT(), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

audio_features_schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("type", StringType(), True),
    StructField("id", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("track_href", StringType(), True),
    StructField("analysis_url", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("time_signature", IntegerType(), True)
])




In [13]:
playlist_df = spark.read.schema(playlist_schema).json(DATASET_FILE, multiLine=True)
slice_df = spark.read.schema(playlist_schema).json(SMALL_SLICE_FLIE, multiLine=True)
# slice_df = spark.read.schema(playlist_schema).json(LITTLE_SLICE_FILE, multiLine=True)
audio_df = spark.read.schema(audio_features_schema).json(SPLITTED_SLICE_AUDIO_FEATURES, multiLine=True) #has less songs than expected

In [14]:
# slice_df.select("tracks").first()

In [15]:
# slice_df.show()

# Item-Based Collaborative Filtering

Item-Based Collaboartive Filtering is the "transpose" approach to user-based CF. This time we won't consider the users' feature vectors, but the items'.
An item's rating vector $\mathbf{r}_i$ is the vector of ratings given to the item $i$ for all the users.

Let $m$ be the number of users, $n$ the number of playlists, then $\mathbf{r}_i \in \mathbb{R}^m$ and $\mathbf{R} \in \mathbb{R}^{m \times n}$.

In order to make a prediction, we take the set of items $I_u$ rated by the user $u$, and we compute the set $I^k_u$ of the top-$k$ most similar items to $i$ rated by $u$, for each item $i \in I_u$. Once done that, we average the $k$ rating vectors weighting them by their respective similarity.

In [16]:
# slice_df = slice_df.cache()

In [17]:
DEBUG = True

In [18]:
NUM_PLAYLISTS = 100_000
SONGS_EMBEDDINGS_PATH = os.path.join(SAVED_DFS_PATH, f"songs_embeddings-train-{NUM_PLAYLISTS}.json")
SONGS_INFO_DF = os.path.join(SAVED_DFS_PATH, f"songs_info_df-train-{NUM_PLAYLISTS}.json") #TODO: Little bug this is songs_df, meaning it hasn't got any info, but we don't actually care.
RATING_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"songs_vector_length-train-{NUM_PLAYLISTS}.txt")
with open(RATING_VECTOR_LENGTH_PATH, "r") as f:
  RATING_VECTOR_LENGTH = int(f.read()) + 1

songs_embeddings = spark.read.schema(playlist_schema_mapped).json(SONGS_EMBEDDINGS_PATH)
songs_df = spark.read.json(SONGS_INFO_DF)

playlist_map_schema = StructType([
    StructField("track_uri", StringType(), True),
    StructField("embedding", VectorUDT(), True)
])
PLAYLIST_MAP_PATH = os.path.join(SAVED_DFS_PATH, f"playlist_map-{NUM_PLAYLISTS}.json")
playlist_map = spark.read.schema(playlist_map_schema).json(PLAYLIST_MAP_PATH)

In [19]:
TRAIN_DF_PATH = os.path.join(SAVED_DFS_PATH, f"train_df-{NUM_PLAYLISTS}.json")
TEST_DF_PATH = os.path.join(SAVED_DFS_PATH, f"test_df-{NUM_PLAYLISTS}.json")

train_df = spark.read.schema(playlist_schema).json(TRAIN_DF_PATH)
test_df = spark.read.schema(playlist_schema).json(TEST_DF_PATH)

In [20]:
def jaccard_similarity(vector_1: SparseVector, vector_2: SparseVector) -> float:
  """
  Computes the Jaccard Similarity between two sparse binary vectors
  """
  # Convert SparseVectors to sets
  set1 = set(vector_1.indices)
  set2 = set(vector_2.indices)

  # Calculate the intersection and union of the sets
  intersection = len(set1.intersection(set2))
  union = len(set1.union(set2))

  # Calculate the similarity
  similarity = intersection / union

  return similarity

In [21]:
jaccard_similarity_udf = F.udf(jaccard_similarity)

def all_against_all_similarity(playlist_map: DataFrame) -> DataFrame:
  # Cross join the DataFrame with itself to get all combinations of tracks
  joined_df = playlist_map.alias('df1').crossJoin(playlist_map.alias('df2'))

  # Apply the Jaccard similarity UDF to calculate the similarity
  similarity_df = joined_df.withColumn('similarity', jaccard_similarity_udf('df1.embedding', 'df2.embedding'))

  # Select the required columns and return the DataFrame
  result_df = similarity_df.select('df1.track_uri', F.col('df2.track_uri').alias('other_uri'), 'similarity')
  return result_df

aga_similarity = all_against_all_similarity(playlist_map)

In [22]:
# ALL_AGAINST_ALL_PATH = os.path.join(SAVED_DFS_PATH, f"all_against_all-{NUM_PLAYLISTS}.parquet")
# # aga_similarity.filter((F.col("similarity") != 1) & (F.col("similarity") != 0)).show()
# aga_similarity.write.parquet(ALL_AGAINST_ALL_PATH)

The similarity between each couple is intractable, we can cluster the similar tracks in buckets using Locally Sensitive Hashing.

In [26]:
from pyspark.ml.feature import BucketedRandomProjectionLSH, MinHashLSH
from pyspark.ml.feature import MinHashLSHModel

LSH_MODEL_PATH = os.path.join(SAVED_DFS_PATH, f"lsh_model-{NUM_PLAYLISTS}.pickle")
if os.path.exists(LSH_MODEL_PATH):
  model = MinHashLSHModel.load(LSH_MODEL_PATH)
else:
  mh = MinHashLSH(inputCol="embedding", outputCol="hashes", numHashTables=5)
  model = mh.fit(playlist_map)
  model.save(LSH_MODEL_PATH)

model.transform(playlist_map).show()

+--------------------+--------------------+--------------------+
|           track_uri|           embedding|              hashes|
+--------------------+--------------------+--------------------+
|spotify:track:000...|(100001,[28824,60...|[[5.82911703E8], ...|
|spotify:track:000...|(100001,[60352],[...|[[9.89451973E8], ...|
|spotify:track:000...|(100001,[76926],[...|[[1.770007514E9],...|
|spotify:track:000...|(100001,[79686],[...|[[2.94025684E8], ...|
|spotify:track:000...|(100001,[14092,21...|[[4.72716012E8], ...|
|spotify:track:000...|(100001,[74372],[...|[[1.791877645E9],...|
|spotify:track:000...|(100001,[8758],[1...|[[8.00841359E8], ...|
|spotify:track:000...|(100001,[70399],[...|[[6.00979172E8], ...|
|spotify:track:001...|(100001,[9796,169...|[[1.60811649E8], ...|
|spotify:track:001...|(100001,[3698,702...|[[1.3893202E7], [...|
|spotify:track:001...|(100001,[92210],[...|[[8.92207576E8], ...|
|spotify:track:001...|(100001,[7110,250...|[[2.05286493E8], ...|
|spotify:track:001...|(10

In [60]:
def transform_to_aggregate(df: DataFrame) -> DataFrame:
  """
  Transforms the dataframe in order to be compatible for the union with an aggregate df.
  """
  return df.withColumn("sum", F.col("distCol")).withColumnRenamed("distCol", "squared_sum")

def merge_aggregate_df(aggregate_df: DataFrame, other_df: DataFrame) -> DataFrame:
  """
  Sums an aggregate df (track_uri, embedding, hashes, squared_sum, sum)
  with a simple df (track_uri, embedding, hashes, distCol).
  """
  other_df = transform_to_aggregate(other_df)
  merged_df = aggregate_df.unionAll(other_df)
  return merged_df.groupBy("track_uri", "embedding", "hashes").agg(
    F.sum(F.pow("squared_sum", 2)).alias("squared_sum"),
    F.sum("sum").alias("sum"))

if DEBUG:
  key = playlist_map.first()[1]
  neigh = model.approxNearestNeighbors(playlist_map, key, 10)
  merged_df_2 = merge_aggregate_df(transform_to_aggregate(neigh), neigh)
  merged_df_2.show() 

In [ ]:
#TODO: this takes too much when I do recommendations.show()
import time
def extract_similar_songs(playlist_tracks: DataFrame, model, k=10) -> DataFrame:
  aggregate_df = None
  tracks_embedding = playlist_tracks.join(playlist_map, "track_uri")
  for row in tqdm(tracks_embedding.collect(), desc='Extracting k-neighbors'):
    k_neigh = model.approxNearestNeighbors(playlist_map, row["embedding"], k).cache()

    t1 = time.time()
    k_neigh.show() #DEBUG to see how much it takes
    t2 = time.time()

    print(f"K-neigh in: {t2-t1}s")
    if aggregate_df == None:
      aggregate_df = transform_to_aggregate(k_neigh).cache()
    else:
      aggregate_df = merge_aggregate_df(aggregate_df, k_neigh).cache()
    
    t1 = time.time()
    aggregate_df.show() #DEBUG to see how much it takes
    t2 = time.time()
    print(f"Aggregation in: {t2-t1}s")

    k_neigh.unpersist()
  return aggregate_df

if DEBUG:
  first_playlist = train_df.limit(1).select(F.explode("tracks")).select("col.*").distinct()
  recommendations = extract_similar_songs(first_playlist, model)

In [64]:
import time

# TODO: THIS MAY BE FASTER???
def extract_similar_songs(playlist_tracks: DataFrame, model, k=10) -> DataFrame:
    tracks_embedding = playlist_tracks.join(playlist_map, "track_uri")

    # Cache the tracks_embedding DataFrame
    tracks_embedding.cache()

    # Collect the embeddings as a list
    embeddings = [row["embedding"] for row in tracks_embedding.collect()]

    k_neigh_dfs = []

    # Perform the approxNearestNeighbors operation for each embedding
    for embedding in tqdm(embeddings):
        k_neigh_df = model.approxNearestNeighbors(playlist_map, embedding, k).cache()
        k_neigh_dfs.append(k_neigh_df)

    # Aggregate the individual k_neigh DataFrames
    if len(k_neigh_dfs) > 0:
        aggregate_df = transform_to_aggregate(k_neigh_dfs[0]).cache()
        for i in tqdm(range(1, len(k_neigh_dfs))):
            aggregate_df = merge_aggregate_df(aggregate_df, k_neigh_dfs[i]).cache()
            aggregate_df.show()
    else:
        aggregate_df = None

    # Show the aggregate DataFrame
    # aggregate_df.show()

    # Unpersist the cached DataFrames
    tracks_embedding.unpersist()
    for k_neigh_df in k_neigh_dfs:
        k_neigh_df.unpersist()

    return aggregate_df


if DEBUG:
    first_playlist = (
        train_df.limit(1).select(F.explode("tracks")).select("col.*").distinct()
    )
    recommendations = extract_similar_songs(first_playlist, model)

  0%|          | 0/141 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

+--------------------+--------------------+--------------------+-------------------+-------------------+
|           track_uri|           embedding|              hashes|        squared_sum|                sum|
+--------------------+--------------------+--------------------+-------------------+-------------------+
|spotify:track:1d2...|  (100001,[1],[1.0])|[[1.020538491E9],...|                0.0|                0.0|
|spotify:track:454...|  (100001,[1],[1.0])|[[1.020538491E9],...|                0.0|                0.0|
|spotify:track:6cO...|(100001,[1,5324,1...|[[3.78384097E8], ...|                0.0|                0.0|
|spotify:track:00F...|  (100001,[1],[1.0])|[[1.020538491E9],...|                0.0|                0.0|
|spotify:track:6D1...|  (100001,[1],[1.0])|[[1.020538491E9],...|                0.0|                0.0|
|spotify:track:4Tc...|(100001,[1,5324,1...|[[2.44792099E8], ...| 0.0330578512396694|0.18181818181818177|
|spotify:track:3vG...|  (100001,[1],[1.0])|[[1.02053849

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

# Old stuff

In [58]:
def transform_to_aggregate(df: DataFrame) -> DataFrame:
  """
  Transforms the dataframe in order to be compatible for the union with an aggregate df.
  """
  return df.withColumn("sum", F.col("similarity")).withColumnRenamed("similarity", "squared_sum")

def merge_aggregate_df(aggregate_df: DataFrame, other_df: DataFrame) -> DataFrame:
  """
  Sums an aggregate df (track_uri, embedding, hashes, squared_sum, sum)
  with a simple df (track_uri, embedding, hashes, distCol).
  """
  other_df = transform_to_aggregate(other_df)
  merged_df = aggregate_df.unionAll(other_df)
  return merged_df.groupBy("track_uri", "embedding").agg(
    F.sum(F.pow("squared_sum", 2)).alias("squared_sum"),
    F.sum("sum").alias("sum"))

In [ ]:
def create_similarity_df(input_vector: SparseVector, playlist_map: DataFrame, similarity_function: Callable) -> DataFrame:
  """
  Returns the similarity df for a single song
  """
  
  @F.udf(returnType=FloatType())
  def compute_similarity(vector1):
    return similarity_function(vector1, input_vector)

  result_df = playlist_map.withColumn("similarity", compute_similarity(playlist_map.embedding))
  
  return result_df

def get_top_k_results(track_uri: str, similarity_df: DataFrame, k: int = 20) -> DataFrame:
  return similarity_df.filter((F.col("similarity") > 0) & (F.col("track_uri") != track_uri)).orderBy(F.col("similarity").desc()).limit(k)

if DEBUG:
  first_song_vector = playlist_map.select("embedding").limit(1).first()[0]
  first_track_uri = playlist_map.select("track_uri").limit(1).first()[0]
  similarity_df = create_similarity_df(first_song_vector, playlist_map, jaccard_similarity)
  top_k_results = get_top_k_results(first_track_uri, similarity_df).cache()
  top_k_results.show()

In [33]:
def extract_similar_songs(playlist_tracks: DataFrame, k=10) -> DataFrame:
  aggregate_df = None
  tracks_embedding = playlist_tracks.join(playlist_map, "track_uri")
  for row in tqdm(tracks_embedding.collect(), desc='Extracting k-neighbors'):
    k_neigh = get_top_k_results(
        track_uri=row.track_uri,
        similarity_df= create_similarity_df(input_vector=row.embedding,
                                            playlist_map=playlist_map,
                                            similarity_function=jaccard_similarity),
        k = 10)
    
    k_neigh.show()
    if aggregate_df == None:
      aggregate_df = transform_to_aggregate(k_neigh).cache()
    else:
      aggregate_df = merge_aggregate_df(aggregate_df, k_neigh).cache()
    
    aggregate_df.show()
  return aggregate_df

if DEBUG:
  first_playlist = train_df.limit(1).select(F.explode("tracks")).select("col.*")
  recommendations = extract_similar_songs(first_playlist, model)

Extracting k-neighbors:   0%|          | 0/141 [00:00<?, ?it/s]

+--------------------+--------------------+----------+
|           track_uri|           embedding|similarity|
+--------------------+--------------------+----------+
|spotify:track:31F...|(100001,[1,5324,1...|       1.0|
|spotify:track:08d...|(100001,[1,5324,1...|       1.0|
|spotify:track:6cO...|(100001,[1,5324,1...|       1.0|
|spotify:track:1J8...|(100001,[1,5324,1...|       1.0|
|spotify:track:2a9...|(100001,[1,5324,1...| 0.8181818|
|spotify:track:4Tc...|(100001,[1,5324,1...| 0.8181818|
|spotify:track:4Y8...|(100001,[1,5324,1...|      0.75|
|spotify:track:2Hr...|(100001,[1,5324,1...| 0.6923077|
|spotify:track:58M...|(100001,[1,5324,1...| 0.6923077|
|spotify:track:30J...|(100001,[1,1382,5...| 0.6923077|
+--------------------+--------------------+----------+

+--------------------+--------------------+-----------+---------+
|           track_uri|           embedding|squared_sum|      sum|
+--------------------+--------------------+-----------+---------+
|spotify:track:31F...|(100001,[

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [31]:
recommendations.show()

Py4JJavaError: ignored

In [ ]:
# def extract_similar_songs(playlist: DataFrame, playlist_map: DataFrame) -> Dict[str, float]:
#   """
#   For each track in the playlist, it extracts the top-k most similar results.
#   """
#   weighted_dict = {}
  
#   # F.udf(returnType=IntegerType())
#   def update_weighted_dict(track_uri: str, similarity: float):
#     nonlocal weighted_dict
#     if track_uri not in weighted_dict:
#       weighted_dict[track_uri] = (similarity, 1)
#     else:
#       curr_similarity = weighted_dict[track_uri][0]
#       count = weighted_dict[track_uri][1]
#       weighted_dict[track_uri] = (curr_similarity + similarity, count+1)
  
#   tracks = playlist.select(F.explode("tracks.track_uri")).withColumnRenamed("col", "track_uri")
#   tracks_mapped = tracks.join(playlist_map, "track_uri")

#   for track_row in tqdm(tracks_mapped.collect(), desc=f"Analyzing tracks"):
#     similarity_df = create_similarity_df(track_row.embedding, playlist_map, jaccard_similarity)
#     top_k_results = get_top_k_results(first_track_uri, similarity_df).cache()
#     top_k_results.foreach(lambda row: update_weighted_dict(row.track_uri, row.similarity))
#     top_k_results.unpersist()
  
#   return weighted_dict

# if DEBUG:
#   first_playlist = train_df.limit(1).cache()
#   weighted_dict = extract_similar_songs(first_playlist, playlist_map)

In [ ]:
def create_similarity_df(playlist: DataFrame, playlist_map: DataFrame, similarityFunction: Callable):
  # Get the current playlist's tracks
  playlist_tracks = playlist.select(F.explode("tracks.track_uri")).withColumnRenamed("col", "track_uri")
  playlist_tracks.show()
  # Let's extract the rating vector for each single track
  playlist_tracks_mapped = playlist_tracks.join(playlist_map, "track_uri")
  playlist_tracks_mapped.show()


  @F.udf(returnType=FloatType())
  def compute_similarity(input_vector: SparseVector, other_vector: SparseVector):
    return jaccard_similarity(input_vector, other_vector)

  # for row in playlist_tracks_mapped.collect():
  similarity_df = playlist_map.withColumn("similarity", compute_similarity(playlist_tracks_mapped.limit(2).embedding, playlist_map.embedding))
  # break
  
  similarity_df.show()
  return

first_playlsit = train_df.limit(1).cache()
create_similarity_df(first_playlsit, playlist_map, jaccard_similarity)

In [ ]:
first_playlsit.show()